In [1]:
import copy
import math

# 外皮情報LV3からLV4へのコンバート

## Outline

- 外皮性能計算プログラムに、外皮の用途別情報を追加したものより、内壁等（建物内部の壁、天井、床）の面積の推定および熱貫流率の決定を行う。
- 外皮の用途別情報が入力された場合において床下空間に属する土間床等面積が0㎡より大きい場合、室と床下空間の間の床の面積の合計は、床下空間に属する土間床等面積の合計（ただし、室用途別の床面積と方位が「下面」の部位面積の差分の合計を上限とする）に等しいとみなす。室と床下空間の間の床は、室用途別の床面積と方位が「下面」の部位面積の差分に応じて、各用途に割り当てる。
- 室と床下空間の間の床は、仕様を想定して与える（合板12mm、参考：SimHeatでの熱負荷計算用の無断熱の層構成）。
- 内壁等の面積は、室の周長を求める際の係数（アスペクト比）と階高を仮定して、外皮の用途別情報および用途別床面積より推定する。
- 内壁等の熱貫流率は一般的な仕様を想定して与える（石膏ボード12.5mm－空気層－石膏ボード12.5mm、参考：SimHeatでの熱負荷計算用の無断熱の層構成）。室内戸は考慮しない。
- その他は、内壁の用途別情報が得られている場合と同様に求める。

## 1. Common Element

#### Outline

'Common'要素はそのまま維持する。

参照：InputDataDefinition.ipynb

#### Function

In [2]:
def convert_common(d):
    return d['Common']

## 2. Walls Element

#### Outline

'Walls'要素はそのまま維持する。

#### Function

In [3]:
def convert_wall(d):
    return d['Walls']

## 3. Windows Element

#### Outline

'Windows'要素はそのまま維持する。

#### Function

In [4]:
def convert_window(d):
    return d['Windows']

## 4. Doors Element

#### Outline

'Doors'要素はそのまま維持する。

#### Function

In [5]:
def convert_door(d):
    return d['Doors']

## 5. Heatbridges Element

#### Outline

'Heatbridges'要素はそのまま維持する。

#### Function

In [6]:
def convert_heatbridge(d):
    return d['Heatbridges']

## 6. EarthfloorPerimeters Element

#### Outline

'EarthfloorPerimeters'要素はそのまま維持する。

#### Function

In [7]:
def convert_earthfloorperimeter(d):
    return d['EarthfloorPerimeters']

## 7. Earthfloors Element

#### Outline

'Earthfloors'要素はそのまま維持する。

#### Function

In [8]:
def convert_earthfloor(d):
    return d['EarthFloors']

## 8. InnerWalls Element

## 8.1. 用途別床面積を取得する関数

In [9]:
def get_area_floor_tuple(d_common):
    return d_common['MainOccupantRoomFloorArea'], \
           d_common['OtherOccupantRoomFloorArea'], \
           d_common['TotalFloorArea'] - d_common['MainOccupantRoomFloorArea'] - d_common['OtherOccupantRoomFloorArea']

In [10]:
def get_area_floor_dictionary(d_common):
    return { 'Main': d_common['MainOccupantRoomFloorArea'],
             'Other': d_common['OtherOccupantRoomFloorArea'], \
             'Nonliving': d_common['TotalFloorArea'] - d_common['MainOccupantRoomFloorArea'] - d_common['OtherOccupantRoomFloorArea']
            }

## 8.2. InnerWalls Element - Ground Floor 

### 1) 層構成

#### Outline

以下の層構成とする。

| 材料 | 厚さ, m | 熱伝導率, W m<sup>-1</sup> K<sup>-1</sup> | 熱抵抗, m<sup>2</sup> k W<sup>-1</sup> | 容積比熱, J L<sup>-1</sup> K<sup>-1</sup> |
|---|---|---|---|
| 合板 | 0.012 | 0.16 | 0.075 | 720.0 |

#### Function

In [11]:
def get_inner_floor_layers():
    return [{'Name': 'PED', 'Cond': 0.16, 'Thick': 0.012, 'SpecH': 720.0}]

### 2) 床下空間上の床面積の推定

#### Outline  
- 外皮の用途別情報が入力された場合において床下空間に属する土間床等面積が0㎡より大きい場合、室と床下空間の間の床の面積の合計は、床下空間に属する土間床等面積の合計（ただし、室用途別の床面積と方位が「下面」の部位面積の差分の合計を上限とする）に等しいとみなす。  
- 室と床下空間の間の床は、室用途別の床面積と方位が「下面」の部位面積の差分に応じて、各用途に割り当てる。

$\sum_{i}^{}A_{b,i}>0$ かつ $A_{dif,MR} + A_{dif,OR} + A_{dif,NR}>0$ の場合

　　$\sum_{i}^{}A_{b,i}≦A_{dif,MR} + A_{dif,OR} + A_{dif,NR}$ の場合

$$
\displaystyle
A_{f,MR}= \sum_{i}^{}A_{b,i} \times \frac{ A_{dif,MR} }{ A_{dif,MR} + A_{dif,OR} + A_{dif,NR} }
$$

$$
\displaystyle
A_{f,OR}= \sum_{i}^{}A_{b,i} \times \frac{ A_{dif,OR} }{ A_{dif,MR} + A_{dif,OR} + A_{dif,NR} }
$$

$$
\displaystyle
A_{f,NR}= \sum_{i}^{}A_{b,i} \times \frac{ A_{dif,NR} }{ A_{dif,MR} + A_{dif,OR} + A_{dif,NR} }
$$

　　$\sum_{i}^{}A_{b,i}>A_{dif,MR} + A_{dif,OR} + A_{dif,NR}$ の場合

$$
\displaystyle
A_{f,MR}= A_{dif,MR}, A_{f,OR}= A_{dif,OR}, A_{f,NR}= A_{dif,NR}
$$

$A_{b,i}$: 床下空間に属する土間床等の部位$i$の面積, m<sup>2</sup>  
$A_{f,MR}$: 主たる居室と床下空間の間の床の面積, m<sup>2</sup>  
$A_{f,OR}$: その他の居室と床下空間の間の床の面積, m<sup>2</sup>  
$A_{f,NR}$: 非居室と床下空間の間の床の面積, m<sup>2</sup>  
$A_f$: 室と床下空間の間の床面積の合計, m<sup>2</sup>  
$A_{dif,MR}$: 主たる居室において、方位が「下面」の部位の面積と土間床等の面積の合計と、床面積の差, m<sup>2</sup>  
$A_{dif,OR}$: その他の居室において、方位が「下面」の部位の面積と土間床等の面積の合計と、床面積の差, m<sup>2</sup>  
$A_{dif,NR}$: 非居室において、方位が「下面」の部位の面積と土間床等の面積の合計と、床面積の差, m<sup>2</sup>  

$$
\displaystyle
A_{dif, MR} = max(0, A_{MR} - \sum_i A_{of,MR,i} - \sum_i A_{ef,MR,i})
$$

$$
\displaystyle
A_{dif, OR} = max(0, A_{OR} - \sum_i A_{of,OR,i} - \sum_i A_{ef,OR,i})
$$

$$
\displaystyle
A_{dif, NR} = max(0, A_{NR} - \sum_i A_{of,NR,i} - \sum_i A_{ef,NR,i})
$$

$A_{MR}$: 主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$: その他の居室の床面積, m<sup>2</sup>  
$A_{NR}$: 非居室の床面積, m<sup>2</sup>   
$A_{of,MR,i}$: 方位が「下面」である主たる居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{of,OR,i}$: 方位が「下面」であるその他の居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{of,NR,i}$: 方位が「下面」である非居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{ef,MR,i}$: 主たる居室の土間床$i$の面積, m<sup>2</sup>  
$A_{ef,OR,i}$: その他の居室の土間床$i$の面積, m<sup>2</sup>  
$A_{ef,NR,i}$: 非居室の土間床$i$の面積, m<sup>2</sup>  

#### Function

In [12]:
# 居室の方位が「下面」である外皮の部位と土間床等の面積の合計
def get_A_of(d, s):
    # s : space name

    # 居室の方位が「下面」である外皮の部位の面積
    A_of_Bottom = sum( y['Area'] for y in d['Walls'] + d['Windows'] + d['Doors'] if y['Space'] == s and y['Direction'] == 'Bottom' )

    # 土間床等の面積の合計
    A_of_EarthFloor = sum( y['Area'] for y in d['EarthFloors'] if ('EarthFloors' in d ) == True and y['Space'] == s )

    return A_of_Bottom + A_of_EarthFloor

In [13]:
# 床下空間に属する土間床等の面積の取得
def get_A_b(d):
    return sum(earthfloor['Area'] for earthfloor in d['EarthFloors'] if earthfloor['Space'] == 'underfloor')

#### Function 1

In [14]:
def get_area_floor_over_underfloor_tuple(d):
    
    # 用途別床面積の取得
    A_MR, A_OR, A_NR = get_area_floor_tuple(d['Common'])
    
    # 下面（direction:Bottom）の壁・窓・ドア面積と土間床等の面積の合計値の取得
    A_of_MR, A_of_OR, A_of_NR = get_A_of(d, 'Main'), get_A_of(d, 'Other'), get_A_of(d, 'Nonliving')
    
    # 方位が「下面」の部位の面積と土間床等の面積の合計値と、床面積の差
    A_dif_MR, A_dif_OR, A_dif_NR = max(0.0, A_MR - A_of_MR), max(0.0, A_OR - A_of_OR), max(0.0, A_NR - A_of_NR)

    A_dif = A_dif_MR + A_dif_OR + A_dif_NR

    # 床下空間に属する土間床等の面積の取得
    A_b = get_A_b(d)
    
    # 室と床下空間の間の床面を設定する場合
    if A_b > 0 and A_dif > 0 :    
        # 居室と床下空間との間の床の面積
        if A_b <= A_dif :
            A_f_MR, A_f_OR, A_f_NR = A_b * A_dif_MR / A_dif, A_b * A_dif_OR / A_dif, A_b * A_dif_NR / A_dif
        else :
            A_f_MR, A_f_OR, A_f_NR = A_dif_MR, A_dif_OR, A_dif_NR
        return True, A_f_MR, A_f_OR, A_f_NR
    # 室と床下空間の間の床面を設定しない場合
    else :
        return False, None, None, None

#### Function 2

In [15]:
def get_area_floor_over_underfloor_dictionary(d):

    # 用途別床面積の取得
    A = get_area_floor_dictionary(d['Common'])
    
    # 下面（direction:Bottom）の壁・窓・ドア面積と土間床等の面積の合計値の取得
    A_of= {x : get_A_of(d, x) for x in ['Main', 'Other', 'Nonliving']}
    
    # 方位が「下面」の部位の面積と土間床等の面積の合計値と、床面積の差
    A_dif = {x : max(0.0, A[x] - A_of[x]) for x in ['Main', 'Other', 'Nonliving']}
    
    A_dif_sum = sum(A_dif[x] for x in A_dif.keys())

    # 床下空間に属する土間床等の面積の取得
    A_b = get_A_b(d)

    #  室と床下空間の間の床面積の推定
    A_f = min(A_dif_sum, A_b)    

    # 室と床下空間の間の床面を設定する場合
    if A_b > 0 and A_dif_sum > 0 :    
        # 居室と床下空間との間の床の面積
        if A_b <= A_dif_sum :
            A_f = {x : A_b * A_dif[x] / A_dif_sum for x in ['Main', 'Other', 'Nonliving']}
        else :
            A_f = {x : A_dif[x] for x in ['Main', 'Other', 'Nonliving']}
        return True, A_f
    # 室と床下空間の間の床面を設定しない場合
    else :
        return False, None

#### Example

In [16]:
d = {
    'Common': {
        'Region': 6,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'Area': 60, 'Direction': 'Top', 'name': 'Ceiling', 'Space': 'Main', 'structure': 'wood', 'Type': 'Ceiling' },
        { 'Area': 15, 'Direction': 'Bottom', 'name': 'Floor', 'Space': 'Other', 'structure': 'wood', 'Type': 'Floor' },
    ],
    'Windows': [
        { 'Area': 30, 'Direction': 'SW', 'name': 'WindowSW', 'Space': 'Main', 'type': 'single' },
        { 'Area': 75, 'Direction': 'Bottom', 'name': 'WindowBottom', 'Space': 'Main', 'type': 'single' },
    ],
    'Doors': [
        { 'Area': 15, 'Direction': 'NW', 'name': 'DoorNW', 'Space': 'Nonliving' },
        { 'Area': 15, 'Direction': 'NE', 'name': 'DoorNE', 'Space': 'Other' }
    ],
    'EarthFloors': [
        { 'Area': 5, 'name': 'other', 'Space': 'underfloor' },
        { 'Area': 5, 'name': 'entrance', 'Space': 'underfloor' }
    ]
}

print(get_area_floor_over_underfloor_tuple(d))
print(get_area_floor_over_underfloor_dictionary(d))

(True, 0.0, 6.0, 4.0)
(True, {'Main': 0.0, 'Other': 6.0, 'Nonliving': 4.0})


### 3) 室と床下空間の間の床の構築

#### Function 1

In [17]:
def get_innerwall_groundfloor_tuple(d):
    
    # 室と床下空間の間の床面積の推定
    IsInnerwallGroundfloor, A_if_MR, A_if_OR, A_if_NR = get_area_floor_over_underfloor_tuple(d)
    
    # 室と床下空間の間の床面を設定する場合
    if IsInnerwallGroundfloor == True :    
        # 壁体名称、部位面積を更新し、「壁体が属する空間種類」「隣接空間種類」を追加する
        ifs = []

        if A_if_MR > 0 :
            ifs.extend([{
                'Name' : 'GroundFloor_Main',
                'Type' : 'GroundFloor',
                'Direction' : 'Horizontal',
                'Area' : A_if_MR,
                'Layers' : get_inner_floor_layers(),
                'Spaces' : ['Main','Underfloor'],
                }])

        if A_if_OR > 0 :
            ifs.extend([ {
                'Name' : 'GroundFloor_Other',
                'Type' : 'GroundFloor',
                'Direction' : 'Horizontal',
                'Area' : A_if_OR,
                'Layers' : get_inner_floor_layers(),
                'Spaces' : ['Other','Underfloor'],
                }])    

        if A_if_NR > 0 :
            ifs.extend([{
                'Name' : 'GroundFloor_Nonliving',
                'Type' : 'GroundFloor',
                'Direction' : 'Horizontal',
                'Area' : A_if_NR,
                'Layers' : get_inner_floor_layers(),
                'Spaces' : ['Nonliving','Underfloor'],
                }])        
        
        return ifs
    # 室と床下空間の間の床面を設定しない場合
    else :
        return []

#### Function 2

In [18]:
def get_innerwall_groundfloor_dictionary(d):
    
    # 室と床下空間の間の床面積の推定
    IsInnerwallGroundfloor, areas_innerfloor = get_area_floor_over_underfloor_dictionary(d)
        
    # 室と床下空間の間の床面を設定する場合
    if IsInnerwallGroundfloor == True :    
        # 壁体名称、部位面積を更新し、「壁体が属する空間種類」「隣接空間種類」を追加する
        ifs = []
        
        for space in ['Main', 'Other', 'Nonliving'] :
            if areas_innerfloor[space] > 0 :
                ifs.extend([ {
                    'Name' : 'GroundFloor_' + space,
                    'Type' : 'GroundFloor',
                    'Direction' : 'Horizontal',
                    'Area' : areas_innerfloor[space],
                    'Layers' : get_inner_floor_layers(),
                    'Spaces' : [space,'Underfloor'],}])

        return ifs
    # 室と床下空間の間の床面を設定しない場合
    else :
        return []

## 8.3. InnerWalls Element - Inner Floor

### 1) 層構成

#### Outline

以下の層構成とする。

| 材料 | 厚さ, m | 熱伝導率, W m<sup>-1</sup> K<sup>-1</sup> | 熱抵抗, m<sup>2</sup> k W<sup>-1</sup> | 容積比熱, J L<sup>-1</sup> K<sup>-1</sup> |
|---|---|---|---|
| 合板 | 0.012 | 0.16 | 0.075 | 720.0 |

#### Function

In [19]:
def get_inner_floor_layers():
    return [{'Name': 'PED', 'Cond': 0.16, 'Thick': 0.012, 'SpecH': 720.0}]

### 2) 室上の床面積の推定

#### Outline

$$
\displaystyle
A_{if,MR}= \max \left(0, A_{MR} -  \left(A_{gf,MR} + \sum_i A_{of,MR,i} + \sum_i A_{ef,MR,i} \right)\right)
$$

$$
\displaystyle
A_{if,OR}= \max \left(0, A_{OR} -  \left(A_{gf,OR} + \sum_i A_{of,OR,i} + \sum_i A_{ef,OR,i} \right)\right)
$$

$$
\displaystyle
A_{if,NR}= \max \left(0, A_{NR} -  \left(A_{gf,NR} + \sum_i A_{of,NR,i} + \sum_i A_{ef,NR,i} \right)\right)
$$

$A_{if,MR}$: その他の居室または非居室との間の、主たる居室の床の面積, m<sup>2</sup>  
$A_{if,OR}$: 主たる居室または非居室との間の、その他の居室の床の面積, m<sup>2</sup>  
$A_{if,NR}$: 主たる居室またはその他の居室との間の、非居室の床の面積, m<sup>2</sup>  
$A_{MR}$: 主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$: その他の居室の床面積, m<sup>2</sup>  
$A_{NR}$: 非居室の床面積, m<sup>2</sup>   
$A_{gf,MR}$: 主たる居室と床下空間の間の床の面積, m<sup>2</sup>  
$A_{gf,OR}$: その他の居室と床下空間の間の床の面積, m<sup>2</sup>  
$A_{gf,NR}$: 非居室と床下空間の間の床の面積, m<sup>2</sup>  
$A_{of,MR,i}$: 方位が「下面」である主たる居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{of,OR,i}$: 方位が「下面」であるその他の居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{of,NR,i}$: 方位が「下面」である非居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{ef,MR,i}$: 主たる居室の土間床$i$の面積, m<sup>2</sup>  
$A_{ef,OR,i}$: その他の居室の土間床$i$の面積, m<sup>2</sup>  
$A_{ef,NR,i}$: 非居室の土間床$i$の面積, m<sup>2</sup>   

#### Function

In [20]:
# 床下空間上の床と、方位が「下面」である外皮の部位と、土間床等の面積の合計
def get_A_f(d, s, innerwalls):
    # s : space name

    # 床下空間上の床の面積
    A_f_GroundFloor = sum( y['Area'] for y in innerwalls if (y['Type'] == 'GroundFloor'and y['Spaces'] == [s,'Underfloor']))

    # 方位が「下面」である外皮の部位の面積
    A_f_Bottom = sum( y['Area'] for y in d['Walls'] + d['Windows'] + d['Doors'] if y['Space'] == s and y['Direction'] == 'Bottom' )

    # 土間床等の面積
    A_f_EarthFloor =  sum( y['Area'] for y in d['EarthFloors'] if ('EarthFloors' in d ) == True and y['Space'] == s )

    return A_f_GroundFloor + A_f_Bottom + A_f_EarthFloor

#### Function 1

In [21]:
def get_area_floor_over_room_tuple(d, innerwalls):
       
    # 用途別床面積の取得
    A_MR, A_OR, A_NR = get_area_floor_tuple(d['Common'])
    
    # 床下空間上の床面積と下面（Direction:Bottom）の壁・窓・ドア面積と土間床等の面積の合計値の取得
    A_f_MR, A_f_OR, A_f_NR = get_A_f(d, 'Main', innerwalls), get_A_f(d, 'Other', innerwalls), get_A_f(d, 'Nonliving', innerwalls)
    
    # 床下空間上の床面積と方位が「下面」の部位の面積と土間床等の面積の合計値と、床面積の差を、室上の床面積とみなす。
    A_if_MR, A_if_OR, A_if_NR = max(0.0, A_MR - A_f_MR), max(0.0, A_OR - A_f_OR), max(0.0, A_NR - A_f_NR)
    
    return A_if_MR, A_if_OR, A_if_NR

#### Function 2

In [22]:
def get_area_floor_over_room_dictionary(d, innerwalls):
    
    # 用途別床面積の取得
    A = get_area_floor_dictionary(d['Common'])
    
    # 床下空間上の床面積と下面（Direction:Bottom）の壁・窓・ドア面積と土間床等の面積の合計値の取得
    A_f = {x : get_A_f(d, x, innerwalls) for x in ['Main', 'Other', 'Nonliving']}
    
    # 床下空間上の床面積と方位が「下面」の部位の面積と土間床等の面積の合計値と、床面積の差を、室上の床面積とみなす。
    A_if = {x : max(0.0, A[x] - A_f[x]) for x in ['Main', 'Other', 'Nonliving']}
    
    return A_if

#### Example

In [23]:
d = {
    'Common': {
        'Region': 6,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'Area': 60, 'Direction': 'Top', 'name': 'Ceiling', 'Space': 'Main', 'structure': 'wood', 'type': 'Ceiling' },
        { 'Area': 15, 'Direction': 'Bottom', 'name': 'Floor', 'Space': 'Other', 'structure': 'wood', 'type': 'Floor' },
    ],
    'Windows': [
        { 'Area': 30, 'Direction': 'SW', 'name': 'WindowSW', 'Space': 'Main', 'type': 'single' },
        { 'Area': 75, 'Direction': 'Bottom', 'name': 'WindowBottom', 'Space': 'Main', 'type': 'single' },
    ],
    'Doors': [
        { 'Area': 15, 'Direction': 'NW', 'name': 'DoorNW', 'Space': 'Nonliving' },
        { 'Area': 15, 'Direction': 'NE', 'name': 'DoorNE', 'Space': 'Other' }
    ],
    'EarthFloors': [
        { 'Area': 5, 'name': 'other', 'Space': 'Underfloor' },
        { 'Area': 5, 'name': 'entrance', 'Space': 'Underfloor' }
    ]
}

innerwalls = [
        { 'Area': 60, 'Name': 'GroundFloor_Main', 'Type' : 'GroundFloor', 'Spaces': ['Main','Underfloor'], 'Direction' : 'Horizontal', 'Layers' : get_inner_floor_layers()},
        { 'Area': 15, 'Name': 'GroundFloor_Other', 'Type' : 'GroundFloor', 'Spaces': ['Other','Underfloor'], 'Direction' : 'Horizontal', 'Layers' : get_inner_floor_layers()},
]

print(get_area_floor_over_room_tuple(d, innerwalls))
print(get_area_floor_over_room_dictionary(d, innerwalls))

(0.0, 30.0, 30.0)
{'Main': 0.0, 'Other': 30.0, 'Nonliving': 30.0}


### 3) 室下の天井面積の推定

#### Outline

$$
\displaystyle
A_{ic,MR}= \max \left(0, A_{MR} -  \sum_i A_{oc,MR,i} \right)
$$

$$
\displaystyle
A_{ic,OR}= \max \left(0, A_{OR} -  \sum_i A_{oc,OR,i} \right)
$$

$$
\displaystyle
A_{ic,NR}= \max \left(0, A_{NR} -  \sum_i A_{oc,NR,i} \right)
$$

$A_{ic,MR}$: その他の居室または非居室との間の、主たる居室の天井の面積, m<sup>2</sup>  
$A_{ic,OR}$: 主たる居室または非居室との間の、その他の居室の天井の面積, m<sup>2</sup>  
$A_{ic,NR}$: 主たる居室またはその他の居室との間の、非居室の天井の面積, m<sup>2</sup>  
$A_{MR}$: 主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$: その他の居室の床面積, m<sup>2</sup>  
$A_{NR}$: 非居室の床面積, m<sup>2</sup>   
$A_{oc,MR,i}$: 方位が「上面」である主たる居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{oc,OR,i}$: 方位が「上面」であるその他の居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{oc,NR,i}$: 方位が「上面」である非居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  

#### Function

In [24]:
# 方位が「上面」である外皮の部位の面積の合計
def get_A_of(d, s):
    # s : space name
    return sum( y['Area'] for y in d['Walls'] + d['Windows'] + d['Doors'] if y['Space'] == s and y['Direction'] == 'Top' )

#### Function 1

In [25]:
def get_area_ceiling_below_room_tuple(d):
    
    # 用途別床面積の取得
    A_MR, A_OR, A_NR = get_area_floor_tuple(d['Common'])
    
    # 上面（Direction:Top）の壁・窓・ドア面積の合計値の取得
    A_oc_MR, A_oc_OR, A_oc_NR = get_A_of(d, 'Main'), get_A_of(d, 'Other'), get_A_of(d, 'Nonliving')
    
    # 方位が「上面」の部位の面積の合計値と、床面積の差
    A_ic_MR, A_ic_OR, A_ic_NR = max(0.0, A_MR - A_oc_MR), max(0.0, A_OR - A_oc_OR), max(0.0, A_NR - A_oc_NR)
   
    return A_ic_MR, A_ic_OR, A_ic_NR

#### Function 2

In [26]:
def get_area_ceiling_below_room_dictionary(d):
    
    # 用途別床面積の取得
    A = get_area_floor_dictionary(d['Common'])
    
    # 上面（Direction:Top）の壁・窓・ドア面積の合計値の取得
    A_oc = {x : get_A_of(d, x) for x in ['Main', 'Other', 'Nonliving']}
    
    # 方位が「上面」の部位の面積の合計値と、床面積の差
    A_ic = {x : max(0.0, A[x] - A_oc[x])for x in ['Main', 'Other', 'Nonliving']}
   
    return A_ic

#### Example

In [27]:
d = {
    'Common': {
        'Region': 6,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'Area': 60, 'Direction': 'Top', 'name': 'Ceiling', 'Space': 'Other', 'structure': 'wood', 'Type': 'Ceiling' },
        { 'Area': 15, 'Direction': 'Bottom', 'name': 'Floor', 'Space': 'Other', 'structure': 'wood', 'Type': 'Floor' },
    ],
    'Windows': [
        { 'Area': 30, 'Direction': 'SW', 'name': 'WindowSW', 'Space': 'Main', 'type': 'single' },
        { 'Area': 75, 'Direction': 'Bottom', 'name': 'WindowBottom', 'Space': 'Main', 'type': 'single' },
    ],
    'Doors': [
        { 'Area': 15, 'Direction': 'NW', 'name': 'DoorNW', 'Space': 'Nonliving' },
        { 'Area': 15, 'Direction': 'NE', 'name': 'DoorNE', 'Space': 'Other' }
    ],
    'EarthFloors': [
        { 'Area': 5, 'name': 'other', 'Space': 'Underfloor' },
        { 'Area': 5, 'name': 'entrance', 'Space': 'Underfloor' }
    ]
}

print(get_area_ceiling_below_room_tuple(d))
print(get_area_ceiling_below_room_dictionary(d))

(30.0, 0.0, 30.0)
{'Main': 30.0, 'Other': 0.0, 'Nonliving': 30.0}


### 4) 室間の床と天井の面積の推定

#### Outline

$$
\displaystyle
A_{if,MR,OR} = \left \{
\begin{array}{ll}
0 & ( A_{ic,OR} = 0 ) \\
\frac{A_{ic,OR}}{A_{ic,OR}+A_{ic,NR}} \times A_{if,MR} & ( A_{ic,OR} > 0 ) \\
\end{array}
\right.
$$
$$
\displaystyle
A_{if,MR,NR} = \left \{
\begin{array}{ll}
0 & ( A_{ic,NR} = 0 ) \\
\frac{A_{ic,NR}}{A_{ic,OR}+A_{ic,NR}} \times A_{if,MR} & ( A_{ic,NR} > 0 ) \\
\end{array}
\right.
$$

$$
\displaystyle
A_{if,OR,MR} = \left \{
\begin{array}{ll}
0 & ( A_{ic,MR} = 0 ) \\
\frac{A_{ic,MR}}{A_{ic,MR}+A_{ic,NR}} \times A_{if,OR} & ( A_{ic,MR} > 0 ) \\
\end{array}
\right.
$$
$$
\displaystyle
A_{if,OR,NR} = \left \{
\begin{array}{ll}
0 & ( A_{ic,NR} = 0 ) \\
\frac{A_{ic,NR}}{A_{ic,MR}+A_{ic,NR}} \times A_{if,OR} & ( A_{ic,NR} > 0 ) \\
\end{array}
\right.
$$

$$
\displaystyle
A_{if,NR,MR} = \left \{
\begin{array}{ll}
0 & ( A_{ic,MR} = 0 ) \\
\frac{A_{ic,MR}}{A_{ic,MR}+A_{ic,OR}} \times A_{if,NR} & ( A_{ic,MR} > 0 ) \\
\end{array}
\right.
$$
$$
\displaystyle
A_{if,NR,OR} = \left \{
\begin{array}{ll}
0 & ( A_{ic,OR} = 0 ) \\
\frac{A_{ic,OR}}{A_{ic,MR}+A_{ic,OR}} \times A_{if,NR} & ( A_{ic,OR} > 0 ) \\
\end{array}
\right.
$$

$A_{if,MR,OR}$: その他の居室との間の、主たる居室の床の面積, m<sup>2</sup>  
$A_{if,MR,NR}$: 非居室との間の、主たる居室の床の面積, m<sup>2</sup>  
$A_{if,OR,MR}$: 主たる居室との間の、その他の居室の床の面積, m<sup>2</sup>  
$A_{if,OR,NR}$: 非居室との間の、その他の居室の床の面積, m<sup>2</sup>  
$A_{if,NR,MR}$: 主たる居室との間の、非居室の床の面積, m<sup>2</sup>  
$A_{if,NR,OR}$: その他の居室との間の、非居室の床の面積, m<sup>2</sup>  
$A_{ic,MR}$: その他の居室または非居室との間の、主たる居室の天井の面積, m<sup>2</sup>  
$A_{ic,OR}$: 主たる居室または非居室との間の、その他の居室の天井の面積, m<sup>2</sup>  
$A_{ic,NR}$: 主たる居室またはその他の居室との間の、非居室の天井の面積, m<sup>2</sup>  
$A_{if,MR}$: その他の居室または非居室との間の、主たる居室の床の面積, m<sup>2</sup>  
$A_{if,OR}$: 主たる居室または非居室との間の、その他の居室の床の面積, m<sup>2</sup>  
$A_{if,NR}$: 主たる居室またはその他の居室との、間の非居室の床の面積, m<sup>2</sup>  

$$
\displaystyle
A_{ic,MR,OR} = \left \{
\begin{array}{ll}
0 & ( A_{if,OR} = 0 ) \\
\frac{A_{if,OR}}{A_{if,OR}+A_{if,NR}} \times A_{ic,MR} & ( A_{if,OR} > 0 ) \\
\end{array}
\right.
$$
$$
\displaystyle
A_{ic,MR,NR} = \left \{
\begin{array}{ll}
0 & ( A_{if,NR} = 0 ) \\
\frac{A_{if,NR}}{A_{if,OR}+A_{if,NR}} \times A_{ic,MR} & ( A_{if,NR} > 0 ) \\
\end{array}
\right.
$$

$$
\displaystyle
A_{ic,OR,MR} = \left \{
\begin{array}{ll}
0 & ( A_{if,MR} = 0 ) \\
\frac{A_{if,MR}}{A_{if,MR}+A_{if,NR}} \times A_{ic,OR} & ( A_{if,MR} > 0 ) \\
\end{array}
\right.
$$
$$
\displaystyle
A_{ic,OR,NR} = \left \{
\begin{array}{ll}
0 & ( A_{if,NR} = 0 ) \\
\frac{A_{if,NR}}{A_{if,MR}+A_{if,NR}} \times A_{ic,OR} & ( A_{if,NR} > 0 ) \\
\end{array}
\right.
$$

$$
\displaystyle
A_{ic,NR,MR} = \left \{
\begin{array}{ll}
0 & ( A_{if,MR} = 0 ) \\
\frac{A_{if,MR}}{A_{if,MR}+A_{if,OR}} \times A_{ic,NR} & ( A_{if,MR} > 0 ) \\
\end{array}
\right.
$$
$$
\displaystyle
A_{ic,NR,OR} = \left \{
\begin{array}{ll}
0 & ( A_{if,OR} = 0 ) \\
\frac{A_{if,OR}}{A_{if,MR}+A_{if,OR}} \times A_{ic,NR} & ( A_{if,OR} > 0 ) \\
\end{array}
\right.
$$

$A_{ic,MR,OR}$: その他の居室との間の、主たる居室の天井の面積, m<sup>2</sup>  
$A_{ic,MR,NR}$: 非居室との間の、主たる居室の天井の面積, m<sup>2</sup>  
$A_{ic,OR,MR}$: 主たる居室との間の、その他の居室の天井の面積, m<sup>2</sup>  
$A_{ic,OR,NR}$: 非居室との間の、その他の居室の天井の面積, m<sup>2</sup>  
$A_{ic,NR,MR}$: 主たる居室との間の、非居室の天井の面積, m<sup>2</sup>  
$A_{ic,NR,OR}$: その他の居室との間の、非居室の天井の面積, m<sup>2</sup>  
$A_{if,MR}$: その他の居室または非居室との間の、主たる居室の床の面積, m<sup>2</sup>  
$A_{if,OR}$: 主たる居室または非居室との間の、その他の居室の床の面積, m<sup>2</sup>  
$A_{if,NR}$: 主たる居室またはその他の居室との、間の非居室の床の面積, m<sup>2</sup>  
$A_{ic,MR}$: その他の居室または非居室との間の、主たる居室の天井の面積, m<sup>2</sup>  
$A_{ic,OR}$: 主たる居室または非居室との間の、その他の居室の天井の面積, m<sup>2</sup>  
$A_{ic,NR}$: 主たる居室またはその他の居室との間の、非居室の天井の面積, m<sup>2</sup>  

#### Function 1

In [28]:
def get_area_slab_between_rooms_tuple(d, innerwalls):
    
    def get_A_if_dictionary(s_m, s_n, A_if_m, A_ic_n, A_ic_o) :
        return ([
            {
                'Name' : 'InnerFloor' + s_m + s_n,
                'Area' : A_if_m * A_ic_n / (A_ic_n + A_ic_o),
                'Space' : s_m,
                'NextSpace' : s_n
            }
        ])
    
    def get_A_ic_dictionary(s_m, s_n, A_ic_m, A_if_n, A_if_o) :
        return ([
            {
                'Name' : 'InnerCeiling' + s_m + s_n,
                'Area' : A_ic_m * A_if_n / (A_if_n + A_if_o),
                'Space' : s_m,
                'NextSpace' : s_n
            }
        ])
    
    # 室上の床面積の取得
    A_if_MR, A_if_OR, A_if_NR = get_area_floor_over_room_tuple(d, innerwalls)
    
    # 室下の天井面積の取得
    A_ic_MR, A_ic_OR, A_ic_NR = get_area_ceiling_below_room_tuple(d)

    # 室上の床面積を、他の用途の室下の天井面積で案分する。
    A_innerfloor=[]
    if A_if_MR > 0 and A_ic_OR > 0 :
        A_innerfloor.extend(get_A_if_dictionary('Main', 'Other', A_if_MR, A_ic_OR, A_ic_NR))
    if A_if_MR > 0 and A_ic_NR > 0 :
        A_innerfloor.extend(get_A_if_dictionary('Main', 'Nonliving', A_if_MR, A_ic_NR, A_ic_OR))
    if A_if_OR > 0 and A_ic_MR > 0 :
        A_innerfloor.extend(get_A_if_dictionary('Other', 'Main', A_if_OR, A_ic_MR, A_ic_NR))
    if A_if_OR > 0 and A_ic_NR > 0 :
        A_innerfloor.extend(get_A_if_dictionary('Other', 'Nonliving', A_if_OR, A_ic_NR, A_ic_MR))
    if A_if_NR > 0 and A_ic_MR > 0 :
        A_innerfloor.extend(get_A_if_dictionary('Nonliving', 'Main', A_if_NR, A_ic_MR, A_ic_OR))
    if A_if_NR > 0 and A_ic_OR > 0 :
        A_innerfloor.extend(get_A_if_dictionary('Nonliving', 'Other', A_if_NR, A_ic_OR, A_ic_MR))

    # 室下の天井面積を、他の用途の室下上の床面積で案分する。
    A_innerceiling=[]
    if A_ic_MR > 0 and A_if_OR > 0 :
        A_innerceiling.extend(get_A_ic_dictionary('Main', 'Other', A_ic_MR, A_if_OR, A_if_NR))
    if A_ic_MR > 0 and A_if_NR > 0 :
        A_innerceiling.extend(get_A_ic_dictionary('Main', 'Nonliving', A_ic_MR, A_if_NR, A_if_OR))
    if A_ic_OR > 0 and A_if_MR > 0 :
        A_innerceiling.extend(get_A_ic_dictionary('Other', 'Main', A_ic_OR, A_if_MR, A_if_NR))
    if A_ic_OR > 0 and A_if_NR > 0 :
        A_innerceiling.extend(get_A_ic_dictionary('Other', 'Nonliving', A_ic_OR, A_if_NR, A_if_MR))
    if A_ic_NR > 0 and A_if_MR > 0 :
        A_innerceiling.extend(get_A_ic_dictionary('Nonliving', 'Main', A_ic_NR, A_if_MR, A_if_OR))
    if A_ic_NR > 0 and A_if_OR > 0 :
        A_innerceiling.extend(get_A_ic_dictionary('Nonliving', 'Other', A_ic_NR, A_if_OR, A_if_MR))
  
    return {'InnerFloors' : A_innerfloor, 'InnerCeilings' : A_innerceiling}

#### Function 2

In [29]:
# 3!通りの室用途の組合せ
def get_list_3_factorial ():
    list3 = [0,1,2]
    
    list_3_factorial = []
    for x in list3 :
        list2 = copy.deepcopy(list3)
        list2.remove(x)
        for y in list2 :
            list1 = copy.deepcopy(list2)
            list1.remove(y) 
            list_3_factorial.extend([[x,y,list1[0]]])
    
    return list_3_factorial

In [30]:
def get_area_slab_between_rooms_dictionary(d, innerwalls):
    
    space = ['Main', 'Other', 'Nonliving']  
    
    # 3!通りの室用途の組合せの取得
    list_3_factorial = get_list_3_factorial()
    
    # 室上の床面積の取得
    A_if =  get_area_floor_over_room_dictionary(d, innerwalls)
    
    # 室下の天井面積の取得
    A_ic =  get_area_ceiling_below_room_dictionary(d)

    # 室上の床面積を、他の用途の室下の天井面積で案分する。
    A_innerfloor = []
    for x in list_3_factorial :
        
        # 自室、隣接室、他の室の設定
        s_m, s_n, s_o = space[x[0]], space[x[1]], space[x[2]]

        if A_if[s_m] > 0 and A_ic[s_n] > 0 :
            A_innerfloor.extend([
                {
                    'Name' : 'InnerFloor' + space[x[0]] + space[x[1]],
                    'Area' : A_if[s_m] * A_ic[s_n] / (A_ic[s_n] + A_ic[s_o]),
                    'Space' : space[x[0]],
                    'NextSpace' : space[x[1]]
                }
            ])

    # 室下の天井面積を、他の用途の室下上の床面積で案分する。
    A_innerceiling = []
    for x in list_3_factorial :
                
        # 自室、隣接室、他の室の設定
        s_m, s_n, s_o = space[x[0]], space[x[1]], space[x[2]]
        
        if A_ic[s_m] > 0 and A_if[s_n] > 0 :
            A_innerceiling.extend([
                {
                    'Name' : 'InnerCeiling' + space[x[0]] + space[x[1]],
                    'Area' : A_ic[s_m] * A_if[s_n] / (A_if[s_n] + A_if[s_o]),
                    'Space' : space[x[0]],
                    'NextSpace' : space[x[1]]
                }
            ])
  
    return {'InnerFloors' : A_innerfloor, 'InnerCeilings' : A_innerceiling}

#### Example

In [31]:
d = {
    'Common': {
        'Region': 6,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'Area': 60, 'Direction': 'Top', 'name': 'Ceiling', 'Space': 'Other', 'structure': 'wood', 'Type': 'Ceiling' },
        { 'Area': 15, 'Direction': 'Bottom', 'name': 'Floor', 'Space': 'Other', 'structure': 'wood', 'Type': 'Floor' },
    ],
    'Windows': [
        { 'Area': 30, 'Direction': 'SW', 'name': 'WindowSW', 'Space': 'Main', 'type': 'single' },
        { 'Area': 75, 'Direction': 'Bottom', 'name': 'WindowBottom', 'Space': 'Main', 'type': 'single' },
    ],
    'Doors': [
        { 'Area': 15, 'Direction': 'NW', 'name': 'DoorNW', 'Space': 'Nonliving' },
        { 'Area': 15, 'Direction': 'NE', 'name': 'DoorNE', 'Space': 'Other' }
    ],
    'EarthFloors': [
        { 'Area': 5, 'name': 'other', 'Space': 'Underfloor' },
        { 'Area': 5, 'name': 'entrance', 'Space': 'Underfloor' }
    ]
}

innerwalls =  [
    { 'Area': 60, 'Name': 'GroundFloor_Main', 'Type' : 'GroundFloor', 'Spaces': ['Main','Underfloor'], 'Direction' : 'Horizontal', 'Layers' : get_inner_floor_layers()},
    { 'Area': 15, 'Name': 'GroundFloor_Other', 'Type' : 'GroundFloor', 'Spaces': ['Other','Underfloor'], 'Direction' : 'Horizontal', 'Layers' : get_inner_floor_layers()},
]

print(get_area_slab_between_rooms_tuple(d, innerwalls))
print(get_area_slab_between_rooms_dictionary(d, innerwalls))

{'InnerFloors': [{'Name': 'InnerFloorOtherMain', 'Area': 15.0, 'Space': 'Other', 'NextSpace': 'Main'}, {'Name': 'InnerFloorOtherNonliving', 'Area': 15.0, 'Space': 'Other', 'NextSpace': 'Nonliving'}, {'Name': 'InnerFloorNonlivingMain', 'Area': 30.0, 'Space': 'Nonliving', 'NextSpace': 'Main'}], 'InnerCeilings': [{'Name': 'InnerCeilingMainOther', 'Area': 15.0, 'Space': 'Main', 'NextSpace': 'Other'}, {'Name': 'InnerCeilingMainNonliving', 'Area': 15.0, 'Space': 'Main', 'NextSpace': 'Nonliving'}, {'Name': 'InnerCeilingNonlivingOther', 'Area': 30.0, 'Space': 'Nonliving', 'NextSpace': 'Other'}]}
{'InnerFloors': [{'Name': 'InnerFloorOtherMain', 'Area': 15.0, 'Space': 'Other', 'NextSpace': 'Main'}, {'Name': 'InnerFloorOtherNonliving', 'Area': 15.0, 'Space': 'Other', 'NextSpace': 'Nonliving'}, {'Name': 'InnerFloorNonlivingMain', 'Area': 30.0, 'Space': 'Nonliving', 'NextSpace': 'Main'}], 'InnerCeilings': [{'Name': 'InnerCeilingMainOther', 'Area': 15.0, 'Space': 'Main', 'NextSpace': 'Other'}, {'Nam

### 5) 室間の床・天井の構築

#### Function

In [32]:
def get_innerwall_innerslab(d, innerwalls):
    
    # 室間の床・天井面積の推定
    areas_innerslab = get_area_slab_between_rooms_dictionary(d, innerwalls)
    
    # 壁体名称、部位面積を更新し、「壁体が属する空間種類」「隣接空間種類」を追加する
    ifs = []
    
    ifs.extend([ {
        'Name' : 'InnerFloor_' + x['Space'],
        'Type' : 'InnerFloor',
        'Direction' : 'Horizontal',
        'Area' : x['Area'],
        'Layers' : get_inner_floor_layers(),
        'Space' : x['Space'], 
        'NextSpace' : x['NextSpace'],
    } for x in areas_innerslab['InnerFloors'] ])
    
    ifs.extend([ {
        'Name' : 'InnerCeiling_' + x['Space'],
        'Type' : 'InnerCeiling',
        'Direction' : 'Horizontal',
        'Area' : x['Area'],
        'Layers' : get_inner_floor_layers(),
        'Space' : x['Space'], 
        'NextSpace' : x['NextSpace'],
    } for x in areas_innerslab['InnerCeilings'] ])
        
    return ifs

## 8.5. InnerWalls Element - Inner Wall 

### 1) 層構成

以下の層構成とする。

| 材料 | 厚さ, m | 熱伝導率, W m<sup>-1</sup> K<sup>-1</sup> | 熱抵抗, m<sup>2</sup>2 k W<sup>-1</sup> | 容積比熱, J L<sup>-1</sup> K<sup>-1</sup> |
|---|---|---|---|
| せっこうボード | 0.0125 | 0.22 | 0.0568 | 830.0 |
| 非密閉空気層 | - | - | 0.090 | 0.0 |
| せっこうボード | 0.0125 | 0.22 | 0.0568 | 830.0 |

#### Function

In [33]:
def get_inner_wall_layers():
    return [
        {'Name': 'GPB', 'Cond': 0.22, 'Thick': 0.0125, 'SpecH': 830.0},
        {'Name': 'GPB', 'Cond': 1/0.09, 'Thick': 1, 'SpecH': 0},
        {'Name': 'GPB', 'Cond': 0.22, 'Thick': 0.0125, 'SpecH': 830.0}
    ]

### 3) 内壁等面積の推定

#### Outline

用途間の内壁等の面積は、各用途の内壁等の面積より決定する。

$$
A_{iw,MR-OR} =\frac{ A_{iw,MR} + A_{iw,OR} - A_{iw,NR} }{2}
$$

$$
A_{iw,MR-NR} = \frac{ A_{iw,MR} + A_{iw,NR} - A_{iw,OR} }{2}
$$

$$
A_{iw,NR-OR} = \frac{ A_{iw,NR} + A_{iw,OR} - A_{iw,MR} }{2}
$$

$ \because $

$$
A_{iw,MR} =A_{iw,MR-OR}+A_{iw,MR-NR}
$$

$$
A_{iw,OR} =A_{iw,MR-OR}+A_{iw,OR-NR}
$$

$$
A_{iw,NR} =A_{iw,MR-OR}+A_{iw,OR-NR}
$$

$A_{iw,MR-OR}$: 主たる居室とその他の居室の間の内壁等の面積, m<sup>2</sup>  
$A_{iw,MR-NR}$: 主たる居室と非居室の間の内壁等の面積, m<sup>2</sup>  
$A_{iw,OR-NR}$: その他の居室と非居室の間の内壁等の面積, m<sup>2</sup>  
$A_{iw,MR}$: 主たる居室の内壁等の面積, m<sup>2</sup>  
$A_{iw,OR}$: その他の居室の内壁等の面積, m<sup>2</sup>  
$A_{iw,NR}$: 非居室の内壁等の面積, m<sup>2</sup>  

各用途の内壁等の面積は、各用途の内壁等の面積の暫定値より、用途間の内壁等の面積が0㎡以上となる範囲で決定する。

$A_{iw,MR,temp}>A_{iw,OR,temp}+A_{iw,NR,temp}$ の場合

　　$A_{OR}=0$ かつ $A_{NR}=0$ の場合

$$A_{iw,MR} = 0, A_{iw,OR} = 0, A_{iw,NR} = 0$$

　　$A_{OR}>0$ または $A_{NR}>0$ で、 $A_{iw,OR,temp}=0$ かつ $A_{iw,NR,temp}=0$ の場合

$$
A_{iw,MR} = A_{iw,MR,temp}
$$
$$
A_{iw,OR} = A_{iw,MR,temp}\times \frac{A_{OR}}{A_{OR}+A_{NR}}
$$
$$
A_{iw,NR} = A_{iw,MR,temp}\times \frac{A_{NR}}{A_{OR}+A_{NR}}
$$

　　$A_{OR}>0$ または $A_{NR}>0$ で、 $A_{iw,OR,temp}>0$ または $A_{iw,NR,temp}>0$ の場合

$$
A_{iw,MR} = A_{iw,MR,temp}
$$
$$
A_{iw,OR} = A_{iw,MR,temp}\times \frac{A_{iw,OR,temp}}{A_{iw,OR,temp}+A_{iw,NR,temp}}
$$
$$
A_{iw,NR} = A_{iw,MR,temp}\times \frac{A_{iw,NR,temp}}{A_{iw,OR,temp}+A_{iw,NR,temp}}
$$

$A_{iw,OR,temp}>A_{iw,MR,temp}+A_{iw,NR,temp}$ の場合

　　$A_{MR}=0$ かつ $A_{NR}=0$ の場合

$$A_{iw,MR} = 0, A_{iw,OR} = 0, A_{iw,NR} = 0$$

　　$A_{OR}>0$ または $A_{NR}>0$ で、 $A_{iw,OR,temp}=0$ かつ $A_{iw,NR,temp}=0$ の場合

$$
A_{iw,OR} = A_{iw,OR,temp}
$$
$$
A_{iw,MR} = A_{iw,OR,temp}\times \frac{A_{MR}}{A_{MR}+A_{NR}}
$$
$$
A_{iw,NR} = A_{iw,OR,temp}\times \frac{A_{NR}}{A_{MR}+A_{NR}}
$$

　　$A_{OR}>0$ または $A_{NR}>0$ で、 $A_{iw,OR,temp}>0$ または $A_{iw,NR,temp}>0$ の場合

$$
A_{iw,OR} = A_{iw,OR,temp}
$$
$$
A_{iw,MR} = A_{iw,OR,temp}\times \frac{A_{iw,MR,temp}}{A_{iw,MR,temp}+A_{iw,NR,temp}}
$$
$$
A_{iw,NR} = A_{iw,OR,temp}\times \frac{A_{iw,NR,temp}}{A_{iw,MR,temp}+A_{iw,NR,temp}}
$$

$A_{iw,NR,temp}>A_{iw,MR,temp}+A_{iw,OR,temp}$ の場合

　　$A_{MR}=0$ かつ $A_{OR}=0$ の場合

$$A_{iw,MR} = 0, A_{iw,OR} = 0, A_{iw,NR} = 0$$

　　$A_{OR}>0$ または $A_{NR}>0$ で、 $A_{iw,OR,temp}=0$ かつ $A_{iw,NR,temp}=0$ の場合

$$
A_{iw,NR} = A_{iw,NR,temp}
$$
$$
A_{iw,MR} = A_{iw,NR,temp}\times \frac{A_{MR}}{A_{MR}+A_{OR}}
$$
$$
A_{iw,OR} = A_{iw,NR,temp}\times \frac{A_{OR}}{A_{MR}+A_{OR}}
$$

　　$A_{OR}>0$ または $A_{NR}>0$ で、 $A_{iw,OR,temp}>0$ または $A_{iw,NR,temp}>0$ の場合

$$
A_{iw,NR} = A_{iw,NR,temp}
$$
$$
A_{iw,MR} = A_{iw,NR,temp}\times \frac{A_{iw,MR,temp}}{A_{iw,MR,temp}+A_{iw,OR,temp}}
$$
$$
A_{iw,OR} = A_{iw,NR,temp}\times \frac{A_{iw,OR,temp}}{A_{iw,MR,temp}+A_{iw,OR,temp}}
$$

上記以外の場合

$$A_{iw,MR} = A_{iw,MR,temp}, A_{iw,OR} = A_{iw,OR,temp}, A_{iw,NR} = A_{iw,NR,temp}$$

$A_{MR}$: 主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$: その他の居室の床面積, m<sup>2</sup>  
$A_{NR}$: 非居室の床面積, m<sup>2</sup>   
$A_{iw,MR,temp}$: 主たる居室の内壁等の面積の暫定値, m<sup>2</sup>  
$A_{iw,OR,temp}$: その他の居室の内壁等の面積の暫定値, m<sup>2</sup>  
$A_{iw,NR,temp}$: 非居室の内壁等の面積の暫定値, m<sup>2</sup>  

各用途の内壁等の面積の暫定値は、室の周長を求める際の係数（アスペクト比）と階高を仮定して、外皮の用途別情報および用途別床面積より推定する。

$$
A_{iw,MR,temp} = \left \{
\begin{array}{ll}
0 & ( A_{MR} = 0 ) \\
A_{w,MR} - A_{ow,MR} = \max \left ( 0,4 \cdot ar_{MR} \cdot h \sqrt {A_{MR}} - A_{ow,MR} \right ) & ( A_{MR} > 0 )
\end{array}
\right.
$$

$$
A_{iw,OR,temp} = \left \{
\begin{array}{ll}
0 & ( A_{OR} = 0 ) \\
A_{w,OR} - A_{ow,OR} = \max \left ( 0, 4 \cdot ar_{NR} \cdot h \sqrt {A_{OR}} - A_{ow,OR} \right ) & ( A_{OR} > 0 )
\end{array}
\right.
$$

$$
A_{iw,NR,temp} = \left \{
\begin{array}{ll}
0 & ( A_{NR} = 0 ) \\
A_{w,NR} - A_{ow,NR} = \max \left ( 0, 4 \cdot ar_{OR} \cdot h \sqrt {A_{NR}} - A_{ow,NR} \right ) & ( A_{NR} > 0 )
\end{array}
\right.
$$

$A_{w,MR}$: 主たる居室の鉛直面の外皮と内壁の面積の合計, m<sup>2</sup>  
$A_{w,OR}$: その他の居室の鉛直面の外皮と内壁の面積の合計, m<sup>2</sup>  
$A_{w,NR}$: 非居室の鉛直面の外皮と内壁の面積の合計, m<sup>2</sup>  
$A_{ow,MR}$: 主たる居室の鉛直面の外皮の面積の合計, m<sup>2</sup>  
$A_{ow,OR}$: その他の居室の鉛直面の外皮の面積の合計, m<sup>2</sup>  
$A_{ow,NR}$: 非居室の鉛直面の外皮の面積の合計, m<sup>2</sup>  
$ar_{MR}$: 主たる居室の周長を求める際の係数, -  
$ar_{OR}$: その他の居室の周長を求める際の係数, -  
$ar_{NR}$: 非居室の周長を求める際の係数, -  
$h$: 階高, m

#### Function

In [34]:
#  室用途別の内壁等の面積
def get_A_ow_for_space(d_ow, s):
    # s : space name       
    return sum(y['Area'] for y in d_ow if (y['Space'] == s and y['Direction'] != 'Top' and y['Direction'] != 'Bottom'))

In [35]:
# 内壁等の面積の暫定値の取得
def get_A_iw_temp(A_f, ar, A_ow):

    h = 2.8 # 階高, m

    return max(0.0, 4 * h * ar * A_f**0.5 - A_ow) if A_f > 0.0 else 0.0

#### Function 1

In [36]:
# 居室の外皮と床下空間との間の床の面積の合計を取得
def get_A_ow_tuple(d):

    # 室用途別の内壁等の面積
    d_ow=[]
    for y in ['Walls', 'Windows', 'Doors']:
        if (y in d) == True :
            d_ow.extend(d[y])

    A_ow_MR, A_ow_OR,A_ow_NR = get_A_ow_for_space(d_ow,'Main'), get_A_ow_for_space(d_ow,'Other'), get_A_ow_for_space(d_ow,'Nonliving')

    return A_ow_MR, A_ow_OR, A_ow_NR

In [37]:
# 内壁等の面積の取得
def get_A_iw_tuple(A_MR, A_OR, A_NR,A_iw_space_MR_temp, A_iw_space_OR_temp, A_iw_space_NR_temp):

    if A_iw_space_MR_temp > A_iw_space_OR_temp + A_iw_space_NR_temp :
        if A_OR == 0 and A_NR == 0 :
            A_iw_space_MR, A_iw_space_OR, A_iw_space_NR = 0, 0, 0
        elif A_iw_space_OR_temp == 0 and A_iw_space_NR_temp == 0 :
            A_iw_space_MR = A_iw_space_MR_temp
            A_iw_space_OR = A_iw_space_MR_temp * A_OR/(A_OR + A_NR)
            A_iw_space_NR = A_iw_space_MR_temp * A_NR/(A_OR + A_NR)
        else :
            A_iw_space_MR = A_iw_space_MR_temp
            A_iw_space_OR = A_iw_space_MR_temp * A_iw_space_OR_temp/(A_iw_space_OR_temp + A_iw_space_NR_temp)
            A_iw_space_NR = A_iw_space_MR_temp * A_iw_space_NR_temp/(A_iw_space_OR_temp + A_iw_space_NR_temp)
            
    elif A_iw_space_OR_temp > A_iw_space_MR_temp + A_iw_space_NR_temp :
        if A_MR == 0 and A_NR == 0 :
            A_iw_space_MR, A_iw_space_OR, A_iw_space_NR = 0, 0, 0
        elif A_iw_space_MR_temp == 0 and A_iw_space_NR_temp == 0 :
            A_iw_space_OR = A_iw_space_OR_temp
            A_iw_space_MR = A_iw_space_OR_temp * A_MR/(A_MR + A_NR)
            A_iw_space_NR = A_iw_space_OR_temp * A_NR/(A_MR + A_NR)
        else :
            A_iw_space_OR = A_iw_space_OR_temp
            A_iw_space_MR = A_iw_space_OR_temp * A_iw_space_MR_temp/(A_iw_space_MR_temp + A_iw_space_NR_temp)
            A_iw_space_NR = A_iw_space_OR_temp * A_iw_space_NR_temp/(A_iw_space_MR_temp + A_iw_space_NR_temp)
            
    elif A_iw_space_NR_temp > A_iw_space_MR_temp + A_iw_space_OR_temp :
        if A_MR == 0 and A_OR == 0 :
            A_iw_space_MR, A_iw_space_OR, A_iw_space_NR = 0, 0, 0
        elif A_iw_space_MR_temp == 0 and A_iw_space_OR_temp == 0 :
            A_iw_space_NR = A_iw_space_NR_temp
            A_iw_space_MR = A_iw_space_NR_temp * A_MR/(A_MR + A_OR)
            A_iw_space_OR = A_iw_space_NR_temp * A_OR/(A_MR + A_OR)
        else :
            A_iw_space_NR = A_iw_space_NR_temp
            A_iw_space_MR = A_iw_space_NR_temp * A_iw_space_MR_temp/(A_iw_space_MR_temp + A_iw_space_OR_temp)
            A_iw_space_OR = A_iw_space_NR_temp * A_iw_space_OR_temp/(A_iw_space_MR_temp + A_iw_space_OR_temp)
            
    else :
        A_iw_space_MR, A_iw_space_OR, A_iw_space_NR = A_iw_space_MR_temp, A_iw_space_OR_temp, A_iw_space_NR_temp

    return A_iw_space_MR, A_iw_space_OR, A_iw_space_NR

In [38]:
# 室用途間の内壁等の面積の推定
# 室用途床面積が0㎡の場合に、室間の内壁等の面積を0㎡とする
def get_A_iw_cor_tuple(A_iw_min, A_iw_s1, A_iw_s2, A_iw_os):
    return max(A_iw_min, (A_iw_s1 + A_iw_s2 - A_iw_os)/2)

In [39]:
def get_area_inner_wall_tuple(d):
    
    # 周長を求める際の係数, -
    ar_MR, ar_OR, ar_NR = 1.2, 1.4, 1.4

    # 用途別床面積の取得
    A_MR, A_OR, A_NR = get_area_floor_tuple(d['Common'])
 
    # 居室の外皮と床下空間との間の床の面積の合計を取得
    A_ow_MR, A_ow_OR, A_ow_NR = get_A_ow_tuple(d)

    # 内壁等の面積の暫定値の取得
    A_iw_space_MR_temp = get_A_iw_temp(A_MR, ar_MR, A_ow_MR)
    A_iw_space_OR_temp = get_A_iw_temp(A_OR, ar_OR, A_ow_OR)
    A_iw_space_NR_temp = get_A_iw_temp(A_NR, ar_NR, A_ow_NR)

    # 内壁等の面積の取得
    A_iw_space_MR, A_iw_space_OR, A_iw_space_NR = get_A_iw_tuple(A_MR, A_OR, A_NR, A_iw_space_MR_temp, A_iw_space_OR_temp, A_iw_space_NR_temp)
    
    # 室間の内壁等の面積の最小値, m2
    A_iw_min_MR_to_OR, A_iw_min_MR_to_NR, A_iw_min_OR_to_NR = 0.0, 0.0, 0.0
    
    # 室用途間の内壁等の面積の推定
    return get_A_iw_cor_tuple(A_iw_min_MR_to_OR, A_iw_space_MR, A_iw_space_OR, A_iw_space_NR), \
           get_A_iw_cor_tuple(A_iw_min_MR_to_NR, A_iw_space_MR, A_iw_space_NR, A_iw_space_OR), \
           get_A_iw_cor_tuple(A_iw_min_OR_to_NR, A_iw_space_OR, A_iw_space_NR, A_iw_space_MR)

#### Function 2

In [40]:
# 居室の外皮と床下空間との間の床の面積の合計を取得
def get_A_ow_dictionary(d):

    # 室用途別の内壁等の面積
    d_ow=[]
    for y in ['Walls', 'Windows', 'Doors']:
        if (y in d) == True :
            d_ow.extend(d[y])

    return { x: get_A_ow_for_space(d_ow, x) for x in ['Main', 'Other', 'Nonliving']}

In [41]:
# 内壁等の面積の取得
def get_A_iw_dictionary(A, A_iw_space_temp):

    A_sum = sum(A[x] for x in A_iw_space_temp.keys())
    A_iw_space_temp_sum = sum(A_iw_space_temp[x] for x in A_iw_space_temp.keys())
    
    A_iw_space = {x:A_iw_space_temp[x] for x in A_iw_space_temp.keys()}
    
    for x in ['Main', 'Other', 'Nonliving'] :   
        
        # 用途間の内壁等の面積が0㎡以上となるよう、各用途の内壁等の面積を調整
        if 2*A_iw_space_temp[x] > A_iw_space_temp_sum :
            
            # 他の2用途の床面積が0㎡の場合は、全用途の内壁等の面積を0㎡とする
            if A[x] == A_sum :
                A_iw_space = {y:0 for y in ['Main', 'Other', 'Nonliving']}
                
            # 他の2用途の内壁等の面積の暫定値が0㎡の場合は、他の2用途の内壁等の面積を、用途別床面積に応じて決定
            elif A_iw_space_temp[x] == A_iw_space_temp_sum :
                for y in ['Main', 'Other', 'Nonliving'] :
                    if x == y :
                        A_iw_space[y] = A_iw_space_temp[y]
                    else :
                        A_iw_space[y] = A_iw_space_temp[x] * A[y]/(A_sum - A[x])
                        
            # 他の2用途の内壁等の面積を、内壁等の面積の暫定値に応じて決定            
            else :
                for y in ['Main', 'Other', 'Nonliving'] :
                    if x == y :
                        A_iw_space[y] = A_iw_space_temp[y]
                    else :
                        A_iw_space[y] = A_iw_space_temp[x] * A_iw_space_temp[y]/(A_iw_space_temp_sum - A_iw_space_temp[x])
                
    return A_iw_space

In [42]:
# 室用途間の内壁等の面積の推定
# 室用途床面積が0㎡の場合に、室間の内壁等の面積を0㎡とする
def get_A_iw_cor_dictionary(A_iw_min, A_iw_s1, A_iw_s2, A_iw_sum):
    return max(A_iw_min, (2*(A_iw_s1 + A_iw_s2) - A_iw_sum)/2)

In [43]:
def get_area_inner_wall_dictionary(d, c):
    # c : correspondence of spaces

    # 周長を求める際の係数, -
    ar = {'Main': 1.2, 'Other': 1.4, 'Nonliving': 1.4}

    # 用途別床面積の取得
    A = get_area_floor_dictionary(d['Common'])
 
    # 居室の外皮と床下空間との間の床の面積の合計を取得
    A_ow = get_A_ow_dictionary(d)

    # 内壁等の面積の取得
    A_iw_space_temp = { x: get_A_iw_temp(A[x], ar[x], A_ow[x]) for x in ['Main', 'Other', 'Nonliving']}
    A_iw_space = get_A_iw_dictionary(A, A_iw_space_temp)
    A_iw_space_sum = sum(A_iw_space[x] for x in A_iw_space.keys())

    # 室間の内壁等の面積の最小値, m2
    A_iw_min = {'main to other': 0.0, 'main to nonliving': 0.0, 'other to nonliving': 0.0}
    
    # 室用途間の内壁等の面積の推定
    return { x: get_A_iw_cor_dictionary(A_iw_min[x], A_iw_space[c[x][0]], A_iw_space[c[x][1]], A_iw_space_sum) for x in c.keys()}

#### Example

In [44]:
correspondence = {
    'main to other': ['Main', 'Other'],
    'main to nonliving': ['Main', 'Nonliving'],
    'other to nonliving': ['Other', 'Nonliving'] 
}  

d = {
    'Common': {
        'Region': 6,
        'MainOccupantRoomFloorArea': 60.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'Area': 60, 'Direction': 'Top', 'name': 'Ceiling', 'Space': 'Main', 'structure': 'wood', 'Type': 'Ceiling' },
        { 'Area': 15, 'Direction': 'Bottom', 'name': 'Floor', 'Space': 'Other', 'structure': 'wood', 'Type': 'Floor' },
    ],
    'Windows': [
        { 'Area': 30, 'Direction': 'SW', 'name': 'WindowSW', 'Space': 'Main', 'type': 'single' },
        { 'Area': 75, 'Direction': 'Bottom', 'name': 'WindowBottom', 'Space': 'Other', 'type': 'single' },
    ],
    'Doors': [
        { 'Area': 15, 'Direction': 'NW', 'name': 'DoorNW', 'Space': 'Nonliving' },
        { 'Area': 15, 'Direction': 'NE', 'name': 'DoorNE', 'Space': 'Other' }
    ],
    'EarthFloors': [
        { 'Area': 40, 'name': 'other', 'Space': 'Underfloor' },
        { 'Area': 20, 'name': 'entrance', 'Space': 'Nonliving' }
    ]
}

print(get_area_inner_wall_tuple(d))
print(get_area_inner_wall_dictionary(d, correspondence))

(106.45675773706458, 0.0, 0.0)
{'main to other': 106.45675773706456, 'main to nonliving': 0.0, 'other to nonliving': 1.4210854715202004e-14}


### 3) 内壁の構築

#### Function 1

In [45]:
def get_innerwall_wall_tuple(d):

    # 内壁面積の推定
    A_iw_MR_OR, A_iw_MR_NR, A_iw_OR_NR = get_area_inner_wall_tuple(d)
    
    # 壁体名称、部位面積を更新し、「壁体が属する空間種類」「隣接空間種類」を追加する
    iws = []

    if A_iw_MR_OR > 0 :
        iws.extend([{
            'Name' : 'Innerwall_main to other',
            'Type' : 'InnerWall',
            'Direction' : 'Vertical',
            'Layers' : get_inner_wall_layers(),
            'Area' : A_iw_MR_OR,
            'Spaces' : ['Main', 'Other']
        }])

    if A_iw_MR_NR > 0 :
        iws.extend([{
            'Name' : 'Innerwall_main to nonliving',
            'Type' : 'InnerWall',
            'Direction' : 'Vertical',
            'Layers' : get_inner_wall_layers(),
            'Area' : A_iw_MR_NR,
            'Spaces' : ['Main', 'Nonliving']
        }])

    if A_iw_OR_NR > 0 :
        iws.extend([{
            'Name' : 'Innerwall_other to nonliving',
            'Type' : 'InnerWall',
            'Direction' : 'Vertical',
            'Layers' : get_inner_wall_layers(),
            'Area' : A_iw_OR_NR,
            'Spaces' : ['Other', 'Nonliving']
        }])

    return iws

#### Function 2

In [46]:
def get_innerwall_wall_dictionary(d):

    # 用途と複数用途間の対応関係の定義
    correspondence = {
        'main to other': ['Main', 'Other'],
        'main to nonliving': ['Main', 'Nonliving'],
        'other to nonliving': ['Other', 'Nonliving'] 
    }  

    # 内壁面積の推定
    areas_innerwall = get_area_inner_wall_dictionary(d, correspondence)
    
    # 壁体名称、部位面積を更新し、「壁体が属する空間種類」「隣接空間種類」を追加する
    iws = []

    for c in correspondence.keys() :
        if areas_innerwall[c] > 0 :
            iws.extend([ {
                'Name' : 'Innerwall_' + c,
                'Type' : 'InnerWall',
                'Direction' : 'Vertical',
                'Layers' : get_inner_wall_layers(),
                'Area' : areas_innerwall[c],
                'Spaces' : correspondence[c]
            }])

    return iws

## 8.3. InnerWalls Element - 統合

#### Function

In [47]:
def convert_innerwall( d ):

    innerwalls =[]
    
    # 室と床下空間の間の床の推定
    innerwalls.extend(get_innerwall_groundfloor_dictionary(d))
    
    # 室間の床・天井の推定
    innerwalls.extend(get_innerwall_innerslab(d, innerwalls))
    
    # 内壁等の推定              
    innerwalls.extend(get_innerwall_wall_dictionary(d))
            
    return innerwalls

#### Example

In [48]:
d = {
    'Common': {'MainOccupantRoomFloorArea': 30.0, 'OtherOccupantRoomFloorArea': 60.0, 'Region': 6, 'TotalFloorArea': 120.0},
    'Doors': [
        {'Area': 0.63, 'Direction': 'NW', 'name': 'DoorNW_main', 'Space': 'Main'},
        {'Area': 1.26, 'Direction': 'NW', 'name': 'DoorNW_other', 'Space': 'Other'},
        {'Area': 0.63, 'Direction': 'NW', 'name': 'DoorNW_nonliving', 'Space': 'Nonliving'},
        {'Area': 0.54, 'Direction': 'NE', 'name': 'DoorNE_main', 'Space': 'Main'},
        {'Area': 1.08, 'Direction': 'NE', 'name': 'DoorNE_other', 'Space': 'Other'},
        {'Area': 0.54, 'Direction': 'NE', 'name': 'DoorNE_nonliving', 'Space': 'Nonliving'}
     ],
    'EarthfloorPerimeters': [
        {'direction': 'NW', 'length': 2.43, 'name': 'other_NW', 'psi': 1.8, 'Space': 'Underfloor'},
        {'direction': 'NE', 'length': 2.43, 'name': 'other_NE', 'psi': 1.8, 'Space': 'Underfloor'}
    ],
    'EarthFloors': [
        {'Area': 5.0, 'name': 'other', 'Space': 'Underfloor'},
        {'Area': 5.0, 'name': 'entrance', 'Space': 'Underfloor'}
    ],
    'Walls': [
        {'Area': 16.95, 'Direction': 'top', 'name': 'Ceiling_main', 'Space': 'Main', 'structure': 'wood', 'Type': 'Ceiling'},
        {'Area': 33.9, 'Direction': 'top', 'name': 'Ceiling_other', 'Space': 'Other', 'structure': 'wood', 'Type': 'Ceiling'},
        {'Area': 16.95, 'Direction': 'top', 'name': 'Ceiling_nonliving', 'Space': 'Nonliving', 'structure': 'wood', 'Type': 'Ceiling'},
        {'Area': 10.1575, 'Direction': 'SW', 'name': 'Wall_SW_main', 'Space': 'Main', 'structure': 'wood', 'Type': 'Wall'},
        {'Area': 20.315, 'Direction': 'SW', 'name': 'Wall_SW_other', 'Space': 'Other', 'structurre': 'wood', 'Type': 'Wall'},
        {'Area': 10.1575, 'Direction': 'SW', 'name': 'Wall_SW_nonliving', 'Space': 'Nonliving', 'structure': 'wood', 'Type': 'Wall'}
    ],
    'Windows': [
        {'Area': 7.5625, 'Direction': 'SW', 'name': 'WindowSW_main', 'Space': 'Main', 'type': 'single'},
        {'Area': 15.125, 'Direction': 'SW', 'name': 'WindowSW_other', 'Space': 'Other', 'type': 'single'},
        {'Area': 7.5625, 'Direction': 'SW', 'name': 'WindowSW_nonliving',  'Space': 'Nonliving', 'type': 'single'},
        {'Area': 0.7925, 'Direction': 'NW', 'name': 'WindowNW_main', 'Space': 'Main', 'type': 'single'},
        {'Area': 1.585, 'Direction': 'NW', 'name': 'WindowNW_other', 'Space': 'Other', 'type': 'single'},
        {'Area': 0.7925, 'Direction': 'NW', 'name': 'WindowNW_nonliving', 'Space': 'Nonliving', 'type': 'single'}
        ]
}

convert_innerwall( d )

[{'Area': 20.0,
  'Direction': 'Horizontal',
  'Layers': [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}],
  'Name': 'InnerFloor_Main',
  'NextSpace': 'Other',
  'Space': 'Main',
  'Type': 'InnerFloor'},
 {'Area': 10.0,
  'Direction': 'Horizontal',
  'Layers': [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}],
  'Name': 'InnerFloor_Main',
  'NextSpace': 'Nonliving',
  'Space': 'Main',
  'Type': 'InnerFloor'},
 {'Area': 30.0,
  'Direction': 'Horizontal',
  'Layers': [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}],
  'Name': 'InnerFloor_Other',
  'NextSpace': 'Main',
  'Space': 'Other',
  'Type': 'InnerFloor'},
 {'Area': 30.0,
  'Direction': 'Horizontal',
  'Layers': [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}],
  'Name': 'InnerFloor_Other',
  'NextSpace': 'Nonliving',
  'Space': 'Other',
  'Type': 'InnerFloor'},
 {'Area': 10.0,
  'Direction': 'Horizontal',
  'Layers': [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 

## 9. Lv4入力情報のコンバート（統合）
- 1)共通情報、2)開口部以外の不透明部位、3)開口部の透明部位、4)開口部の不透明部位、5) 線熱橋、6)土間床等の外周部の線熱橋、7)土間床、8)内壁等の各コンバート関数を統合し、Lv3の入力情報をLv4の入力情報にコンバートする。

### Function

In [49]:
def convert(d):
    d_lv4_input = {}
    d_lv4_input['Common'] = convert_common(d)
    if ('Walls' in d) == True:
        d_lv4_input['Walls'] = convert_wall(d)
    if ('Windows' in d) == True:
        d_lv4_input['Windows'] = convert_window(d)
    if ('Doors' in d) == True:
        d_lv4_input['Doors'] = convert_door(d)
    if ('EarthfloorPerimeters' in d) == True:
        d_lv4_input['EarthfloorPerimeters'] = convert_earthfloorperimeter(d)
    if ('EarthFloors' in d) == True:
        d_lv4_input['EarthFloors'] = convert_earthfloor(d)
    if ('Heatbridges' in d) == True:
        d_lv4_input['Heatbridges'] = convert_heatbridge(d)
    d_lv4_input['InnerWalls'] = convert_innerwall(d)

    return d_lv4_input

### Example

In [50]:
d = {
    'Common': {'MainOccupantRoomFloorArea': 30.0, 'OtherOccupantRoomFloorArea': 60.0, 'Region': 6, 'TotalFloorArea': 120.0},
    'Doors': [
        {'Area': 0.63, 'Direction': 'NW', 'name': 'DoorNW_main', 'Space': 'Main'},
        {'Area': 1.26, 'Direction': 'NW', 'name': 'DoorNW_other', 'Space': 'Other'},
        {'Area': 0.63, 'Direction': 'NW', 'name': 'DoorNW_nonliving', 'Space': 'Nonliving'},
        {'Area': 0.54, 'Direction': 'NE', 'name': 'DoorNE_main', 'Space': 'Main'},
        {'Area': 1.08, 'Direction': 'NE', 'name': 'DoorNE_other', 'Space': 'Other'},
        {'Area': 0.54, 'Direction': 'NE', 'name': 'DoorNE_nonliving', 'Space': 'Nonliving'}
     ],
    'EarthfloorPerimeters': [
        {'Direction': 'NW', 'length': 2.43, 'name': 'other_NW', 'psi': 1.8, 'Space': 'Underfloor'},
        {'Direction': 'NE', 'length': 2.43, 'name': 'other_NE', 'psi': 1.8, 'Space': 'Underfloor'}
    ],
    'EarthFloors': [
        {'Area': 30.0, 'Name': 'Main', 'Space': 'Main'},
        {'Area': 5.0, 'Name': 'Entrance', 'Space': 'Nonliving'}
    ],
    'Walls': [
        {'Area': 16.95, 'Direction': 'top', 'name': 'Ceiling_main', 'Space': 'Main', 'structure': 'wood', 'Type': 'Ceiling'},
        {'Area': 33.9, 'Direction': 'top', 'name': 'Ceiling_other', 'Space': 'Other', 'structure': 'wood', 'Type': 'Ceiling'},
        {'Area': 16.95, 'Direction': 'top', 'name': 'Ceiling_nonliving', 'Space': 'Nonliving', 'structure': 'wood', 'Type': 'Ceiling'},
        {'Area': 10.1575, 'Direction': 'SW', 'name': 'Wall_SW_main', 'Space': 'Main', 'structure': 'wood', 'Type': 'Wall'},
        {'Area': 20.315, 'Direction': 'SW', 'name': 'Wall_SW_other', 'Space': 'Other', 'structure': 'wood', 'Type': 'Wall'},
        {'Area': 10.1575, 'Direction': 'SW', 'name': 'Wall_SW_nonliving', 'Space': 'Nonliving', 'structure': 'wood', 'Type': 'Wall'}
    ],
    'Windows': [
        {'Area': 7.5625, 'Direction': 'SW', 'name': 'WindowSW_main', 'Space': 'Main', 'type': 'single'},
        {'Area': 15.125, 'Direction': 'SW', 'name': 'WindowSW_other', 'Space': 'Other', 'type': 'single'},
        {'Area': 7.5625, 'Direction': 'SW', 'name': 'WindowSW_nonliving',  'Space': 'Nonliving', 'type': 'single'},
        {'Area': 0.7925, 'Direction': 'NW', 'name': 'WindowNW_main', 'Space': 'Main', 'type': 'single'},
        {'Area': 1.585, 'Direction': 'NW', 'name': 'WindowNW_other', 'Space': 'Other', 'type': 'single'},
        {'Area': 0.7925, 'Direction': 'NW', 'name': 'WindowNW_nonliving', 'Space': 'Nonliving', 'type': 'single'}
        ]
}

convert(d)

{'Common': {'MainOccupantRoomFloorArea': 30.0,
  'OtherOccupantRoomFloorArea': 60.0,
  'Region': 6,
  'TotalFloorArea': 120.0},
 'Doors': [{'Area': 0.63,
   'Direction': 'NW',
   'Space': 'Main',
   'name': 'DoorNW_main'},
  {'Area': 1.26, 'Direction': 'NW', 'Space': 'Other', 'name': 'DoorNW_other'},
  {'Area': 0.63,
   'Direction': 'NW',
   'Space': 'Nonliving',
   'name': 'DoorNW_nonliving'},
  {'Area': 0.54, 'Direction': 'NE', 'Space': 'Main', 'name': 'DoorNE_main'},
  {'Area': 1.08, 'Direction': 'NE', 'Space': 'Other', 'name': 'DoorNE_other'},
  {'Area': 0.54,
   'Direction': 'NE',
   'Space': 'Nonliving',
   'name': 'DoorNE_nonliving'}],
 'EarthFloors': [{'Area': 30.0, 'Name': 'Main', 'Space': 'Main'},
  {'Area': 5.0, 'Name': 'Entrance', 'Space': 'Nonliving'}],
 'EarthfloorPerimeters': [{'Direction': 'NW',
   'Space': 'Underfloor',
   'length': 2.43,
   'name': 'other_NW',
   'psi': 1.8},
  {'Direction': 'NE',
   'Space': 'Underfloor',
   'length': 2.43,
   'name': 'other_NE',
   